
  
# Replication project: "A Neural Representation of Sketch Drawings"  
  
Jou-Hui Ho, Hojin Kang  
  
  


### Description of the problem  


The main problem to solve consists of the generation of new hand-drawn sketch drawings from training examples of <i>Quick, Draw!</i>, a sketching game. The solution will be implemented using an encoder-decoder autoregressive model.  
  
- <b>Input:</b> The input of the model is a dataset of hand-drawn sketches, each represented as a sequence of motor actions controlling a pen: the direction of the movement, when to lift the pen up, and when to stop drawing. More concretely, each input is a vector containing 5 elements: <img src="https://latex.codecogs.com/gif.latex?(\Delta&space;x,&space;\Delta&space;y,&space;p_1,&space;p_2,&space;p_3)" title="(\Delta x, \Delta y, p_1, p_2, p_3)" /> 
, where the first two elements are the offset distance from the previous point, and the last 3 elements represents a one-hot vector of the 3 mentioned states. The total input is the concatenation of the direct sequence of vectors and the inverse one.  
  
- <b>Output:</b> At each test, our autoregressive model outputs the 5 element vector’s probability distribution, and then we get samples of that distribution. Hence, out complete model should output a vector of the same structure than the given one at the input.  
  
  

### Metrics  


Most of the results obtained from the work are visual rather than specific metrics. The results are mainly reflected in the similar but distinct sketches generated from the autoregressive model. We expect to generate similar sketches to the training data as the paper shows.  
  
A metric that could be used to measure the network's performance is its loss value. However, these values could widely vary depending on the dataset that is being used and the parameters of the network. It also depends on the training (and the number of training samples used). Hence, we cannot specify an expected value of loss, since it would be more of a wild guess rather than an expected value.  

We propose another metric that could be used to measure the performance of the system. Since what we want is to generate similar sketches to those made by humans, we make a grading system in which each sketch (either real or generated by the algorithm) is graded as real, possibly real or fake. From this grading system we expect the real sketches to be mainly classified as real, whilst most of the ones generated by the algorithm should be classified as either real or possibly real. The expected result is that at least more than half of the sketches generated by the algorithm are classified as real or possibly real, on average.
  
  

### Training data  

The training data is obtained from the  <i>Quick, Draw!</i> dataset, given at https://github.com/googlecreativelab/quickdraw-dataset. It contains 50 million drawings across 345 categories, each captured as timestamped vectors, tagged with the concept that the sketch is representing. Each class is a dataset of 70K training samples, in addition to 2.5K validation and 2.5K test samples. As an initial approach of the project’s aim, we will take 10 categories of the complete dataset.  
  
Ha & Eck’s previous work includes a simplification of the dataset, scaling each data into a 256x256 region, resampling all strokes with a 1 pixel spacing. Each drawing array corresponds to an array of strokes, and each stroke consists of a sequence of vectors of five elements, as we previously described.   
  
  

### Initial architecture  

We will implement a Sequence-to-Sequence Variational Autoencoder (VAE), consisting of two neural networks: an encoder and a decoder. Our encoder is a bidirectional RNN that receives the concatenation of both direct and inverse sequences of the sketch strokes, and passes it through a fully connected layer, resulting 2 parameters $\mu$ and $\sigma$, each of size $N_z$.   

$$\mu = W_\mu h + b_\mu, \quad \hat{\sigma} = W_\sigma h + b_\sigma, \quad \sigma = exp(\frac{\hat{\sigma}}{2})$$
  
Then, using these parameters, along with $\mathcal{N}(0,I)$, we construct a distribution for a latent vector of size $N_z$, given by $z = \mu + \sigma \cdot \mathcal{N}(0,I)$. It is important to note that the outputs of the encoder are the parameters of the gaussian distribution ($\mu$ and $\sigma$), and the next step consists in sampling from that distribution, obtaining a vector of size $N_z$.   
  
Under this encoding scheme, the latent vector z is not deterministic but a random vector condition the input sketch.  
  
The decoder is an autoregressive RNN that takes that latent vector $z$ and samples output sketches. At each step i, the decoder is fed with the previous point $S_{i-1}$ and the latent vector $z$ as a concatenated input $x_i$. The position of each stroke is modeled as a Gaussian Mixture Model, with $M$ normal distributions, and $(q_1, q_2, q_3)$ as a categorical distribution to model $(p_1, p_2, p_3)$.   
  
$$p(\Delta x, \Delta y) = \sum_{j=1}^M \Pi_j \mathcal{N}(\Delta x, \Delta y | \mu_{x,j} , \mu_{y,j} , \sigma_{x,j} , \sigma_{y,j} , \rho_{xy,j})$$, where $\sum_{j=1}^M=1$  
  
The output of the decoder is the concatenation of $6M+3$ parameters: 5M of each ($\mu_{x}$ , $\mu_{y}$ , $\sigma_{x}$ , $\sigma_{y}$, $\rho_{xy}$)  parameters of the $M$ distributions, $M$ parameters due to the mixture weights of each distribution, and the 3 logits needed to generate $(q_1, q_2, q_3)$.   
  
$$y_i = \Big[\big(\Pi_1 \mu_x \mu_y \sigma_x \sigma_y \rho_{xy}\big)_1 \dots \big(\Pi_1 \mu_x \mu_y \sigma_x \sigma_y \rho_{xy}\big)_M \big(\hat{q}_1 \hat{q}_2 \hat{q}_3\big)\Big]$$  
  
Finally, we apply softmax to each type of parameters, and sample a 5 element vector from the output distribution. This final vector is the actual stroke element $S’_i$ for that time step, and it’s used as input for the next time step. And so on, until it returns $p_3=1$, or when it has reached the maximum number of samples.

The following image shows the architecture of the VAE.

![Modelo](images/modelo.jpg)

The following equations characterize the backward enconder of the architecture.

$$h_{0}^{B} = 0$$

$$h_{i}^{B} = f_{i}(S_{n + 1 - i}U_{e}^{B} + h_{i-1}^{B}V_{e}^{B} + b_{e}^{B})$$

$$h_{\leftarrow} = h_{n}^{B}$$

We also have the equations that characterize the forward encoder of the architecture.

$$h_{0}^{F} = 0$$

$$h_{i}^{F} = f_{i}(S_{i}U_{e}^{F} + h_{i-1}^{F}V_{e}^{F} + b_{e}^{F})$$

$$h_{\rightarrow} = h_{n}^{F}$$

Once $h_{\leftarrow}$ and $h_{\rightarrow}$ are calculated, $h$ is the vector resulting from both vectors being concatenated.

$$h = [h_{\leftarrow}, h_{\rightarrow}]$$

From the previous value we obtain $\sigma$ and $\mu$, used to calculate the normal distribution given by $z = \mu + \sigma \cdot \mathcal{N}(0,I)$. This can be done with the following equations.

$$\mu = W_{\mu}h + b_{\mu}$$

$$\hat{\sigma} = W_{\sigma}h + b_{\sigma}$$

$$\sigma = exp(\frac{\hat{\sigma}}{2})$$

 Once the value of $z$ is determined, the values for the decoder are given by the following equations.

$$h_{0} = f_{0}(z)$$

$$h_{i} = f_{i}(zR_{D} + h_{i-1}V_{D} + S_{i-1}U_{D} + b_{D})$$

Then we calculate the value for $y_{i}$ as.

$$y_{i} = W_{y}h_{i} + b_{y}$$

The values for $y_{i}$ correspond to the values for the $M$ Gaussian Mixture Models (GMM) for the next data point, and the type of stroke (lifted pen, pen on paper, or end of sketch). This means that:

$$y_i = \Big[\big(\hat{\Pi}_1 \mu_x \mu_y \hat{\sigma}_x \hat{\sigma}_y \rho_{xy}\big)_1 \dots \big(\hat{\Pi}_1 \mu_x \mu_y \hat{\sigma}_x \hat{\sigma}_y \rho_{xy}\big)_M \big(\hat{q}_1 \hat{q}_2 \hat{q}_3\big)\Big]$$  

With this values, the actual values for the distributions are calculated as follows.

$$\sigma_{x} = exp(\hat{\sigma_{x}})$$

$$\sigma_{y} = exp(\hat{\sigma_{y}})$$

$$\rho_{xy} = f(\hat{\rho}_{xy})$$

$$q_{k} = softmax(\hat{q}_{k})$$

$$\Pi_{k} = softmax(\hat{\Pi}_{k})$$

From the previous expressions, the values for the next stroke can be sampled.



### Github repository: 
https://github.com/hojink1996/proyecto_deep_learning/





```python

```
